In [ ]:
import configparser
import boto3
import awswrangler as wr
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)

config = configparser.ConfigParser()
config.read_file(open('conf.cfg'))

KEY         = config.get('AWS', 'key')
SECRET      = config.get('AWS', 'secret')
REGION      = config.get('AWS', 'region')
LOG_DATA    = config.get('S3', 'LOG_DATA')
SONG_DATA   = config.get('S3', 'SONG_DATA')

session = boto3.Session(region_name=REGION,
                aws_access_key_id=KEY,
                aws_secret_access_key=SECRET)


# print(wr.s3.does_object_exist('s3://udacity-dend/song_data', boto3_session=session))


In [ ]:
df = wr.s3.read_json(path='s3://udacity-dend/log_data', boto3_session=session, dataset= True)

In [ ]:
wr.s3.to_parquet(
    df=df,
    path='s3://udacity-labs-ik/parquet-test/',
    dataset=True,
    partition_cols=['artist']
)

In [ ]:
df.head(5)

In [ ]:
df.dtypes

# PySpark

In [1]:
import configparser
import boto3
import awswrangler as wr
import pandas as pd
import logging
from pyspark.sql import SparkSession
import os

logging.basicConfig(level=logging.INFO)

config = configparser.ConfigParser()
config.read_file(open('conf.cfg'))

KEY         = config.get('AWS', 'key')
SECRET      = config.get('AWS', 'secret')
REGION      = config.get('AWS', 'region')
LOG_DATA    = config.get('S3', 'LOG_DATA')
SONG_DATA   = config.get('S3', 'SONG_DATA')

os.environ["AWS_ACCESS_KEY_ID"]= config.get('AWS', 'key')
os.environ["AWS_SECRET_ACCESS_KEY"]= config.get('AWS', 'secret')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

spark = create_spark_session()

:: loading settings :: url = jar:file:/Users/ivankhokhlov/DataEngineering/Project%204%20-%20Data%20Lake/.venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ivankhokhlov/.ivy2/cache
The jars for the packages stored in: /Users/ivankhokhlov/.ivy2/jars
com.amazonaws#aws-java-sdk-pom added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d97a8588-83fd-4875-bfc2-33ef9fb02990;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk-pom;1.10.34 in central
	found org.apache.hadoop#hadoop-aws;2.7.2 in central
	found org.apache.hadoop#hadoop-common;2.7.2 in central
	found org.apache.hadoop#hadoop-annotations;2.7.2 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.4 in central
	found c

In [4]:
song_data = os.path.join('s3a://udacity-dend/', 'song_data', '*', '*', '*')

In [5]:
df = spark.read.json(song_data)
df.printSchema()

Py4JJavaError: An error occurred while calling o34.json.
: java.lang.NumberFormatException: For input string: "100M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1538)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3303)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSourc

In [ ]:
df = spark.read\
  .option("multiLine", True).option("mode", "PERMISSIVE")\
  .json('s3a://udacity-dend/log_data')
df.printSchema()